In [2]:
#确保每个集有每个类别的数据
#pip install scikit-multilearn
import pandas as pd
import numpy as np
import re
from skmultilearn.model_selection import iterative_train_test_split

# 定义标签列表和创建标签到数字的映射
tags_list = [
    "传感器与仪器",
    "生命健康技术",
    "新材料科技",
    "新能源与动力",
    "数字智能技术",
    "智能制造与装备",
    "其它"   
]
tag2id = {tag: idx for idx, tag in enumerate(tags_list)}

# 读取CSV文件
df = pd.read_csv('result_data_7.csv')  # 修改为你的CSV文件路径
print(df.columns.tolist())  # 查看所有实际列名

# 处理标签列（支持中英文逗号分隔）
def convert_tags(tags_str):
    tags = re.split(r'[，,]+', tags_str)
    tags = [tag.strip() for tag in tags if tag.strip()]
    tag_ids = []
    for tag in tags:
        if tag in tag2id:
            tag_ids.append(str(tag2id[tag]))
        else:
            raise ValueError(f"发现未知标签: '{tag}'，请检查数据一致性")
    return tag_ids

df['tags_ids'] = df['Tags'].apply(convert_tags)

# 拼接文本列并清理
text_columns = ['Title', 'Abstract', 'Keywords']
for col in text_columns:
    df[col] = df[col].fillna('')

df['text'] = df[text_columns].apply(lambda x: ' '.join(x), axis=1)
df['text'] = df['text'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

# 将标签转换为二进制矩阵
y = np.zeros((len(df), len(tags_list)), dtype=int)
for i, tags in enumerate(df['tags_ids']):
    for tag_id in tags:
        y[i, int(tag_id)] = 1

# 使用多标签分层抽样划分数据集
X = df.index.values.reshape(-1, 1)  # 使用索引作为特征

# 第一次划分：70%训练集，30%临时集
X_train, y_train, X_temp, y_temp = iterative_train_test_split(X, y, test_size=0.3)

# 第二次划分：将临时集分为15%验证集和15%测试集
X_val, y_val, X_test, y_test = iterative_train_test_split(X_temp, y_temp, test_size=0.5)

# 获取对应的数据子集
train_df = df.iloc[X_train.flatten()]
val_df = df.iloc[X_val.flatten()]
test_df = df.iloc[X_test.flatten()]

# 保存数据集到TXT文件
def save_dataset(df, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for text, tags in zip(df['text'], df['tags_ids']):
            f.write(f"{text}\t{','.join(tags)}\n")

save_dataset(train_df, 'train.txt')
save_dataset(val_df, 'dev.txt')
save_dataset(test_df, 'test.txt')

# 验证每个子集的标签分布
def check_coverage(df_set, set_name):
    existing_tags = set()
    for tags in df_set['tags_ids']:
        for tag in tags:
            existing_tags.add(int(tag))
    print(f"{set_name}包含标签: {sorted(existing_tags)}")

check_coverage(train_df, "训练集")
check_coverage(val_df, "验证集")
check_coverage(test_df, "测试集")

print("\n处理完成！")
print(f"训练集样本数: {len(train_df)}")
print(f"验证集样本数: {len(val_df)}")
print(f"测试集样本数: {len(test_df)}")

['No', 'Title', 'Abstract', 'Keywords', 'Tags']
训练集包含标签: [0, 1, 2, 3, 4, 5, 6]
验证集包含标签: [0, 1, 2, 3, 4, 5, 6]
测试集包含标签: [0, 1, 2, 3, 4, 5, 6]

处理完成！
训练集样本数: 8902
验证集样本数: 1908
测试集样本数: 1907


In [3]:
#针对新能源与动力与智能制造装备做过滤
import pandas as pd
import numpy as np
import re
from skmultilearn.model_selection import iterative_train_test_split

# ==================== 需要过滤的高频词列表 ====================
HIGH_FREQ_WORDS = {
    '机器', '汽油', '气油', '能', '控', '系', '统', '技', '电', '动', '车', 
    '测', '试', '装', '备', '产', '造', '究', '物', '理', '源', '气', '管', 
    '化', '铁', '金', '钢', '高', '带', '液', '修', '强', '和', '中', '材', 
    '设', '托', '体', '氧', '于', '特', '地', '成', '型', '铝', '该', '光', 
    '继', '抗', '孔', '养', '上', '列', '头', '智', '应', '面', '基', '箱', 
    '自', '台', '人', '属', '空', '瓶', '种', '锁', '收', '片', '口', '米', 
    '联', '频', '断', '息', '计', '别', '活', '验', '瞬', '制', '据', '处', 
    '然', '保', '废', '配', '蒸', '数', '层', '绿', '向', '清', '盐', '压', 
    '加', '号', '发', '质', '池', '油', '氟', '震', '长', '集', '输', '感', 
    '水', '性', '警', '轮', '学', '挡', '建', '轴', '络', '内', '环', '温', 
    '检', '式', '品', '结', '合', '信','项目', '实现', '方法', '技术', '研究', 
    '具有', '提出', '采用', '通过','成果', '所述', '结果', '包括', '提供', 
    '解决', '评价', '中国', '系统','应用', '影响', '形成','问题','重庆市','创新',
    '进行','关键','主要','国家'
}

# ==================== 核心过滤函数 ====================
def remove_high_freq(text):
    """增强文本清洗逻辑"""
    # 移除特殊空白字符
    text = re.sub(r'[\t\n\r]', ' ', text)
    # 过滤高频词
    for word in HIGH_FREQ_WORDS:
        text = text.replace(word, '')
    # 合并连续空格
    return re.sub(r'\s+', ' ', text).strip()

# ==================== 数据加载与预处理 ====================
tags_list = [
    "传感器与仪器",
    "生命健康技术",
    "新材料科技",
    "新能源与动力",
    "数字智能技术",
    "智能制造与装备",
    "其它"   
]
tag2id = {tag: idx for idx, tag in enumerate(tags_list)}

# 读取数据
df = pd.read_csv('result_data_7.csv')

# 标签处理（增加空值过滤）
def convert_tags(tags_str):
    tags = re.split(r'[，,]+', tags_str.strip())
    valid_tags = [str(tag2id[tag]) for tag in tags if tag in tag2id]
    return valid_tags if valid_tags else None  # 返回None用于后续过滤

df['tags_ids'] = df['Tags'].apply(convert_tags)

# 过滤无效数据（重要修复）
print(f"原始数据量: {len(df)}")
df = df.dropna(subset=['tags_ids'])  # 移除无有效标签的样本
df = df[df['tags_ids'].apply(len) > 0]  # 移除标签为空的样本
print(f"有效数据量: {len(df)}")

# ==================== 条件过滤处理 ====================
# 创建需要过滤的标签掩码
mask = df['tags_ids'].apply(lambda tags: any(tag in {'0','3', '5','6'} for tag in tags))

# 文本处理（仅对指定标签进行过滤）
text_columns = ['Title', 'Abstract', 'Keywords']
for col in text_columns:
    df[col] = df[col].fillna('')
    df.loc[mask, col] = df.loc[mask, col].apply(remove_high_freq)

# 合并文本字段并二次清洗
df['text'] = df[text_columns].apply(
    lambda x: re.sub(r'\s+', ' ', ' '.join(x)).strip(), 
    axis=1
)

# ==================== 数据划分 ====================
y = np.zeros((len(df), len(tags_list)), dtype=int)
for i, tags in enumerate(df['tags_ids']):
    for tag in tags:
        y[i, int(tag)] = 1

X = df.index.values.reshape(-1, 1)
X_train, y_train, X_temp, y_temp = iterative_train_test_split(X, y, test_size=0.3)
X_val, y_val, X_test, y_test = iterative_train_test_split(X_temp, y_temp, test_size=0.5)

# ==================== 数据保存（增加格式验证） ====================
def save_dataset(df, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for text, tags in zip(df['text'], df['tags_ids']):
            # 最终格式检查
            if '\t' in text:
                text = text.replace('\t', ' ')  # 确保文本不含制表符
            line = f"{text}\t{','.join(tags)}\n"
            assert line.count('\t') == 1, f"格式错误: {line}"
            f.write(line)

train_df = df.iloc[X_train.flatten()]
val_df = df.iloc[X_val.flatten()]
test_df = df.iloc[X_test.flatten()]

save_dataset(train_df, 'train.txt')
save_dataset(val_df, 'dev.txt')
save_dataset(test_df, 'test.txt')

# ==================== 验证输出 ====================
print("\n处理完成！样本分布：")
print(f"训练集: {len(train_df)}条")
print(f"验证集: {len(val_df)}条")
print(f"测试集: {len(test_df)}条")

# 验证数据格式
def check_file(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if line.count('\t') != 1:
                print(f"格式错误行 {i+1}: {line.strip()}")

print("\n格式验证：")
check_file('train.txt')
check_file('dev.txt')
check_file('test.txt')

原始数据量: 12717
有效数据量: 12717

处理完成！样本分布：
训练集: 8902条
验证集: 1906条
测试集: 1909条

格式验证：


In [1]:
#增加关键词增强
import pandas as pd
import numpy as np
import re
from skmultilearn.model_selection import iterative_train_test_split

# ==================== 高频词过滤配置 ====================
HIGH_FREQ_WORDS = {
    '机器', '汽油', '气油', '能', '控', '系', '统', '技', '电', '动', '车', 
    '测', '试', '装', '备', '产', '造', '究', '物', '理', '源', '气', '管', 
    '化', '铁', '金', '钢', '高', '带', '液', '修', '强', '和', '中', '材', 
    '设', '托', '体', '氧', '于', '特', '地', '成', '型', '铝', '该', '光', 
    '继', '抗', '孔', '养', '上', '列', '头', '智', '应', '面', '基', '箱', 
    '自', '台', '人', '属', '空', '瓶', '种', '锁', '收', '片', '口', '米', 
    '联', '频', '断', '息', '计', '别', '活', '验', '瞬', '制', '据', '处', 
    '然', '保', '废', '配', '蒸', '数', '层', '绿', '向', '清', '盐', '压', 
    '加', '号', '发', '质', '池', '油', '氟', '震', '长', '集', '输', '感', 
    '水', '性', '警', '轮', '学', '挡', '建', '轴', '络', '内', '环', '温', 
    '检', '式', '品', '结', '合', '信','项目', '实现', '方法', '技术', '研究', 
    '具有', '提出', '采用', '通过','成果', '所述', '结果', '包括', '提供', 
    '解决', '评价', '中国', '系统','应用', '影响', '形成','问题','重庆市','创新',
    '进行','关键','主要'
}

# ==================== 关键词替换逻辑 ====================
# 解析keyword.txt构建替换规则
category_keywords = {}
current_category = None
with open('keyword.txt', 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if line.startswith('#'):
            current_category = line[1:]
            category_keywords[current_category] = []
        else:
            if current_category and line:
                category_keywords[current_category].append(line)

# 生成替换字典（关键词 -> 重复后的字符串）
replace_dict = {}
for category, keywords in category_keywords.items():
    # 按长度降序排列避免短词优先匹配
    keywords_sorted = sorted(keywords, key=lambda x: len(x), reverse=True)
    # 根据出现顺序计算重复次数（前4个分别重复4/3/2/1次，后续重复1次）
    for idx, word in enumerate(keywords_sorted):
        repeat = max(8 - idx, 1)
        replace_dict[word] = word * repeat

# 构建正则表达式（按关键词长度降序匹配）
sorted_keywords = sorted(replace_dict.keys(), key=lambda x: -len(x))
keyword_pattern = re.compile('|'.join(map(re.escape, sorted_keywords)))

def keyword_replacer(text):
    """关键词重复替换函数"""
    return keyword_pattern.sub(lambda x: replace_dict[x.group()], text)

# ==================== 核心过滤函数 ====================
def remove_high_freq(text):
    text = re.sub(r'[\t\n\r]', ' ', text)
    for word in HIGH_FREQ_WORDS:
        text = text.replace(word, '')
    return re.sub(r'\s+', ' ', text).strip()

# ==================== 数据加载与预处理 ====================
tags_list = [
    "传感器与仪器",
    "生命健康技术",
    "新材料科技",
    "新能源与动力",
    "数字智能技术",
    "智能制造与装备",
    "其它"   
]
tag2id = {tag: idx for idx, tag in enumerate(tags_list)}

df = pd.read_csv('result_data_7.csv')

# 标签处理
def convert_tags(tags_str):
    tags = re.split(r'[，,]+', tags_str.strip())
    valid_tags = [str(tag2id[tag]) for tag in tags if tag in tag2id]
    return valid_tags if valid_tags else None

df['tags_ids'] = df['Tags'].apply(convert_tags)

# 过滤无效数据
print(f"原始数据量: {len(df)}")
df = df.dropna(subset=['tags_ids'])
df = df[df['tags_ids'].apply(len) > 0]
print(f"有效数据量: {len(df)}")

# ==================== 条件过滤处理 ====================
mask = df['tags_ids'].apply(lambda tags: any(tag in {'3', '5','6'} for tag in tags))

text_columns = ['Title', 'Abstract', 'Keywords']
for col in text_columns:
    df[col] = df[col].fillna('')
    df.loc[mask, col] = df.loc[mask, col].apply(remove_high_freq)

# 合并文本字段
df['raw_text'] = df[text_columns].apply(
    lambda x: re.sub(r'\s+', ' ', ' '.join(x)).strip(), 
    axis=1
)

# 应用关键词替换（在最终文本上处理）
df['text'] = df['raw_text'].apply(keyword_replacer)

# ==================== 数据划分 ====================
y = np.zeros((len(df), len(tags_list)), dtype=int)
for i, tags in enumerate(df['tags_ids']):
    for tag in tags:
        y[i, int(tag)] = 1

X = df.index.values.reshape(-1, 1)
X_train, y_train, X_temp, y_temp = iterative_train_test_split(X, y, test_size=0.3)
X_val, y_val, X_test, y_test = iterative_train_test_split(X_temp, y_temp, test_size=0.5)

# ==================== 数据保存 ====================
def save_dataset(df, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for text, tags in zip(df['text'], df['tags_ids']):
            line = f"{text}\t{','.join(tags)}\n"
            assert line.count('\t') == 1, f"格式错误: {line}"
            f.write(line)

train_df = df.iloc[X_train.flatten()]
val_df = df.iloc[X_val.flatten()]
test_df = df.iloc[X_test.flatten()]

save_dataset(train_df, 'train.txt')
save_dataset(val_df, 'dev.txt')
save_dataset(test_df, 'test.txt')

# ==================== 验证输出 ====================
print("\n处理完成！样本分布：")
print(f"训练集: {len(train_df)}条")
print(f"验证集: {len(val_df)}条")
print(f"测试集: {len(test_df)}条")

def check_file(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if line.count('\t') != 1:
                print(f"格式错误行 {i+1}: {line.strip()}")

print("\n格式验证：")
check_file('train.txt')
check_file('dev.txt')
check_file('test.txt')

原始数据量: 12717
有效数据量: 12717

处理完成！样本分布：
训练集: 8902条
验证集: 1906条
测试集: 1909条

格式验证：
